## 优化与调度：ViT 分层 OneCycleLR 策略

本配置采用 PyTorch 的 **AdamW 优化器**结合 **OneCycleLR 调度器**，实现高效的 ViT 分层微调。

| 模块 | 核心设置 | 策略说明 |
| :--- | :--- | :--- |
| **优化器 (AdamW)** | **分层学习率 (LR)** + `weight_decay=1e-4` | 将参数分为 **`head`** ($\mathbf{10^{-3}}$) 和 **`encoder`** ($\mathbf{10^{-4}}$) 两组，分配不同基础 LR，并应用 AdamW 解耦的权重衰减。 |
| **调度器 (OneCycleLR)** | `max_lr` 列表 + `steps_per_epoch` + `epochs` | 实现**学习率单周期循环**，在整个训练过程中，学习率从低到高再到低平滑变化，加速 ViT 收敛。`max_lr` 列表确保了**分层 LR 比例**在循环峰值保持不变。 |
| **损失函数** | `nn.CrossEntropyLoss()` | 适用于多分类任务的标准交叉熵损失。 |

**要点：** OneCycleLR 的 `step()` 必须在**每批次训练**后调用。

In [ ]:
from torch.optim.lr_scheduler import OneCycleLR
# 前置参数
num_epochs = 200
# 分层学习率
LR_CUSTOM_HEAD = 0.001 #  (自定义分类头，可以稍高，需要根据模型进行微调)
LR_LAST_BLOCK = 0.0001   # (解冻层，一般非常低，为自定义分类头的1/10或1/100)

# 定义损失函数
criterion = nn.CrossEntropyLoss()

# 自定义优化器和学习率调度器
param_groups = [
    # 参数组 1: 自定义分类头 (model.head)
    # 这部分是新初始化的，使用相对较高的 LR
    {'params': model.head.parameters(), 'lr': LR_CUSTOM_HEAD, 'group_name': 'custom_head'},
    
    # 参数组 2: 解冻的最后一个 Transformer 编码器块 (model.encoder.layers[-1])
    # 这部分是预训练的，使用非常低的 LR 进行微调
    {'params': model.encoder.layers[-1].parameters(), 'lr': LR_LAST_BLOCK, 'group_name': 'last_block_fine_tune'}
]

optimizer = optim.AdamW(
    param_groups, 
    weight_decay=1e-4 # 应用于所有参数组
)

steps_per_epoch = len(train_loader) 

# OneCycleLR 的 max_lr 参数：对应优化器中每个参数组的最大学习率
scheduler = OneCycleLR(
    optimizer, 
    max_lr=[LR_CUSTOM_HEAD, LR_LAST_BLOCK], # 顺序必须与 param_groups 保持一致
    steps_per_epoch=steps_per_epoch, 
    epochs=num_epochs
)

print("Successful! Vision Transformer 学习率调度策略已设置。")